In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    # df = df.sample(frac=1).reset_index(drop=True)  # Randomiza as linhas
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [27]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import levenberg_marquardt as lm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error
from keras import regularizers
from keras import initializers

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)


    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)
     
      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
      
      self.set_dataset()
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

    def set_dataset(self):
      self.train_dataset, self.train_input, self.train_output = self.create_dataset(self._train[0], self._train[1])
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(self._val[0], self._val[1])
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(self._test[0], self._test[1])
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(self._vt[0], self._vt[1])

    def create_model(self, input_size=2, sumary=False):
      regularizer = regularizers.L2(0.2)
      initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

      self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(30,
                                activation='tanh',
                                input_shape=(input_size,),
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(16,
                                activation='tanh',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(1,
                                activation='linear',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
        )])

      if (sumary == True):
        self.model.summary()

      self.model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.MeanSquaredError())

      self.lm_model = lm.ModelWrapper(
          tf.keras.models.clone_model(self.model))

      self.lm_model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
          loss=lm.MeanSquaredError())

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics


    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

In [31]:
from keras import saving
from keras import models
import pickle

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)

  def PrepareTraining(self):
    trainer = TrainWithSmallDataset()
    trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
    return (trainer)

  def Train(self, trainer, epochs=1000):
    trainer.create_model()
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test
    trainer.set_dataset()

  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ Dataset [{idx + 1}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

  # brief: for each dataset
  def LoopDataset(self, sort_by, boundarie, load=''):
    for j in range(self.dataset_run_times):
      trainer = self.PrepareTraining()
      if (load):
        self.LoadDataset(load, trainer)
      if (self.LoopWeights(sort_by, boundarie, trainer, j) == True):
        self.SaveDataset(trainer, f"dataset_{j}")
        self.DisplayBetterResults('mse_sup', j)
      self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    excel_path = f'../results/metrics_{dataset}.xlsx'
    df.to_excel(excel_path, index=True)
    print(f"DataFrame salvo em {excel_path}")

# Treinando com 25 dados


In [32]:
tester = Tester(run_times=100, dataset_run_times=100)

In [34]:
tester.LoopDataset(sort_by='mse_sup', boundarie=2.0, load='dataset_12')

+++++++++++ Dataset [1] | 1 ++++++++++++++++++
Elapsed time:  4.1939385020014015
Stopped at epoch:  148
1/1 [==============================] - 0s 21ms/step
+++++++++++ Dataset [1] | 2 ++++++++++++++++++
Elapsed time:  6.365321709999989
Stopped at epoch:  239
1/1 [==============================] - 0s 17ms/step
+++++++++++ Dataset [1] | 3 ++++++++++++++++++
Elapsed time:  6.396995561997755
Stopped at epoch:  239
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [1] | 4 ++++++++++++++++++
Elapsed time:  10.203554465999332
Stopped at epoch:  389
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [1] | 5 ++++++++++++++++++
Elapsed time:  1.3900221459989552
Stopped at epoch:  39
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [1] | 6 ++++++++++++++++++
Elapsed time:  8.466901067000435
Stopped at epoch:  328
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [1] | 7 ++++++++++++++++++
Elapsed time:  25.795

2024-01-31 03:57:38.830573: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  22.91540889099997
Stopped at epoch:  917
1/1 [==============================] - 0s 13ms/step
+++++++++++ Dataset [1] | 30 ++++++++++++++++++
Elapsed time:  0.7112292759993579
Stopped at epoch:  16
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 31 ++++++++++++++++++
Elapsed time:  0.6055776239991246
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 32 ++++++++++++++++++
Elapsed time:  3.479939277000085
Stopped at epoch:  130
1/1 [==============================] - 0s 17ms/step
+++++++++++ Dataset [1] | 33 ++++++++++++++++++
Elapsed time:  1.142864819998067
Stopped at epoch:  32
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 34 ++++++++++++++++++
Elapsed time:  6.997019067002839
Stopped at epoch:  266
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 35 ++++++++++++++++++
Elapsed time:  6.510249085997202
Stopped at epoch:  254
1/1 [=====

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_12,0.945140,0.787233,0.955601,0.840806,0.864369,0.230234,1.422774,0.096625,1.342336,0.719481
model_1_5,0.935453,0.763659,0.937915,0.815237,0.840419,0.270888,1.580413,0.135117,1.557932,0.846524
model_0_12,0.947767,0.756444,0.841682,0.879042,0.871389,0.219211,1.628661,0.344549,1.019926,0.682238
model_0_7,0.920541,0.750355,0.737510,0.831711,0.812403,0.333471,1.669377,0.571260,1.419026,0.995143
model_1_33,0.920872,0.749402,0.844159,0.803361,0.811748,0.332084,1.675748,0.339158,1.658072,0.998615
...,...,...,...,...,...,...,...,...,...,...
model_1_21,0.820220,0.636734,0.411260,0.649352,0.600544,0.754499,2.429163,1.281279,2.956690,2.118985
model_1_32,0.828788,0.633704,0.630665,0.768579,0.740310,0.718540,2.449425,0.803788,1.951355,1.377571
model_1_26,0.813922,0.630429,0.239982,0.617320,0.539951,0.780931,2.471325,1.654034,3.226784,2.440408
model_1_41,0.801196,0.628328,-0.180086,0.686154,0.508490,0.834336,2.485373,2.568231,2.646372,2.607301


DataFrame salvo em ../results/metrics_0.xlsx
+++++++++++ Dataset [2] | 1 ++++++++++++++++++
Elapsed time:  8.87270221200015
Stopped at epoch:  333
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [2] | 2 ++++++++++++++++++
Elapsed time:  25.068955894999817
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [2] | 3 ++++++++++++++++++
Elapsed time:  2.799950271997659
Stopped at epoch:  95
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [2] | 4 ++++++++++++++++++
Elapsed time:  0.792260793998139
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [2] | 5 ++++++++++++++++++
Elapsed time:  5.330384368997329
Stopped at epoch:  200
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [2] | 6 ++++++++++++++++++
Elapsed time:  1.6772739000007277
Stopped at epoch:  49
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [2] | 7 ++

...

DataFrame salvo em ../results/metrics_1.xlsx
+++++++++++ Dataset [3] | 1 ++++++++++++++++++
Elapsed time:  2.890547090999462
Stopped at epoch:  103
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [3] | 2 ++++++++++++++++++
Elapsed time:  0.44943223199879867
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [3] | 3 ++++++++++++++++++
Elapsed time:  4.249764217998745
Stopped at epoch:  155
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [3] | 4 ++++++++++++++++++
Elapsed time:  2.8385062439992907
Stopped at epoch:  96
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [3] | 5 ++++++++++++++++++
Elapsed time:  4.619101819000207
Stopped at epoch:  173
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [3] | 6 ++++++++++++++++++
Elapsed time:  5.634744764996867
Stopped at epoch:  208
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [3] |

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_2_1,0.932817,0.762777,0.772132,0.850978,0.834818,0.281951,1.586310,0.495910,1.256565,0.876238
model_2_16,0.920115,0.755732,0.631748,0.846547,0.802499,0.335258,1.633423,0.801430,1.293929,1.047680
model_2_10,0.933840,0.751024,0.829216,0.853805,0.848775,0.277660,1.664902,0.371678,1.232724,0.802201
model_2_17,0.924848,0.750682,0.708060,0.852522,0.822902,0.315397,1.667192,0.635351,1.243546,0.939448
model_2_14,0.933638,0.744782,0.863761,0.829217,0.836319,0.278507,1.706643,0.296499,1.440055,0.868277
model_2_19,0.918882,0.744454,0.769474,0.807163,0.799449,0.340434,1.708839,0.501695,1.626017,1.063856
model_2_11,0.929204,0.743400,0.759024,0.841972,0.824971,0.297114,1.715887,0.524438,1.332505,0.928472
model_2_9,0.928946,0.741687,0.821165,0.830520,0.828617,0.298200,1.727341,0.389200,1.429064,0.909132
model_2_20,0.914831,0.740248,0.740360,0.803176,0.790308,0.357437,1.736961,0.565057,1.659634,1.112345
model_2_3,0.919807,0.737549,0.606373,0.852142,0.801740,0.336552,1.755011,0.856655,1.246751,1.051703


DataFrame salvo em ../results/metrics_2.xlsx
+++++++++++ Dataset [4] | 1 ++++++++++++++++++
Elapsed time:  1.1898605989990756
Stopped at epoch:  32
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [4] | 2 ++++++++++++++++++
Elapsed time:  0.5380627129998174
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [4] | 3 ++++++++++++++++++
Elapsed time:  8.703746634000709
Stopped at epoch:  330
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [4] | 4 ++++++++++++++++++
Elapsed time:  5.714256914998259
Stopped at epoch:  227
1/1 [==============================] - 0s 13ms/step
+++++++++++ Dataset [4] | 5 ++++++++++++++++++
Elapsed time:  7.102454449999641
Stopped at epoch:  269
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [4] | 6 ++++++++++++++++++
Elapsed time:  1.0728282009986287
Stopped at epoch:  31
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [4] | 

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_3_16,0.925099,0.779649,0.679391,0.849945,0.814973,0.314344,1.473486,0.697745,1.265273,0.981509
model_3_17,0.935454,0.763661,0.937931,0.815236,0.840422,0.270884,1.580402,0.135081,1.557944,0.846512
model_3_2,0.932775,0.762671,0.772219,0.850848,0.834732,0.282127,1.587018,0.495722,1.257660,0.876691
model_3_10,0.925774,0.760577,0.575042,0.886058,0.822269,0.311512,1.601020,0.924840,0.960766,0.942803
model_3_0,0.934574,0.754868,0.800634,0.851211,0.840850,0.274580,1.639202,0.433882,1.254599,0.844240
model_3_13,0.924269,0.752931,0.841467,0.805338,0.812767,0.317827,1.652152,0.345017,1.641404,0.993210
model_3_15,0.926661,0.750151,0.836526,0.814053,0.818680,0.307790,1.670742,0.355769,1.567919,0.961844
model_3_7,0.921877,0.744079,0.795471,0.809770,0.806854,0.327864,1.711344,0.445119,1.604032,1.024575
model_3_14,0.912383,0.741354,0.787681,0.783923,0.784714,0.367711,1.729565,0.462071,1.821975,1.142023
model_3_1,0.927640,0.738498,0.911326,0.811330,0.831860,0.303679,1.748667,0.192981,1.590874,0.891928


DataFrame salvo em ../results/metrics_3.xlsx
+++++++++++ Dataset [5] | 1 ++++++++++++++++++
Elapsed time:  3.9945815169994603
Stopped at epoch:  149
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [5] | 2 ++++++++++++++++++
Elapsed time:  0.4716453729997738
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [5] | 3 ++++++++++++++++++
Elapsed time:  2.173578287001874
Stopped at epoch:  69
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [5] | 4 ++++++++++++++++++
Elapsed time:  22.629005148999568
Stopped at epoch:  889
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [5] | 5 ++++++++++++++++++
Elapsed time:  2.560411694001232
Stopped at epoch:  88
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [5] | 6 ++++++++++++++++++
Elapsed time:  3.742720174999704
Stopped at epoch:  133
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [5] | 